In [10]:
# the imports used in A2
import re
# import json
from glob import glob
import os
from io import StringIO
from itertools import groupby
import pickle

import numpy as np
import bs4
%matplotlib inline
import matplotlib.pyplot as plt
# Imports that might help with various functionality
import functools
import operator
import pickle

# Additional imports from A3
from __future__ import print_function
import math
from collections import defaultdict
from nltk.tokenize import TreebankWordTokenizer
import Levenshtein  # package python-Levenshtein

# Additional imports from A5
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jericahuang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
import sys
# Ensure that your kernel is using Python3
assert sys.version_info.major == 3

In [12]:
# import the json
import json
with open("data3.json", "r") as f:
    women_summaries = json.load(f)

print(len(women_summaries))
# print(women_summaries)
# print(women_summaries[0]['name'])
# print(women_summaries[0]['summary'])

2449


In [18]:
# creates a list of women to keep as a global variable
women_names = list()
women_name_to_data = {}

for i in range(len(women_summaries)):
    name = women_summaries[i]['name']
    end_index = name.find('(')
    if end_index != -1 and name[: (end_index-1)] not in women_names :
        women_names.append(name[: (end_index-1)])
        women_name_to_data[name[: (end_index-1)]] = women_summaries[i]
    elif end_index == -1 and name not in women_names :
        women_names.append(name)
        women_name_to_data[name] = women_summaries[i]
        
print(women_name_to_data["Grace Crowfoot"]["views"])
pickle.dump(women_name_to_data, open( "women_name_to_data.pickle", "wb" ), protocol=2)

4


In [14]:
# creates a dictionary {'name', [list of words in the summary]}
women_dict_1sent = dict()
# for i in range(len(women_summaries)):
#     name = women_summaries[i]['name']
for i in range(len(women_names)):
    name = women_names[i]
    summary = women_summaries[i]['summary']
    # to do: 
    # hardcoded the words that we're looking for
    # change to regex later time
    index_was = summary.find('was ')
    index_is = summary.find('is ')
    index_currently = summary.find('currently ')
    if (index_was == -1 and index_is == -1):
        summary2 = summary[index_currently:]
        # index_period = summary2.find('.')
        # first_sent = summary2[:index_period]
    elif (index_was == -1 and index_currently == -1):
        summary2 = summary[index_is:]
        # index_period = summary2.find('.')
        # first_sent = summary2[:index_period]
    elif (index_is == -1 and index_currently == -1):
        summary2 = summary[index_was:]
        # index_period = summary2.find('.')
        # first_sent = summary2[:index_period]
    else :
        index_min = min(index_is, index_was)
        summary2 = summary[index_min:]
        # index_period = summary2.find('.')
        # first_sent = summary2[:index_period]
    # might want to change if we use ML that finds proper nouns
    # implement a stemmer
    first_sent_lower = summary2.lower()
    tokenizer = TreebankWordTokenizer()
    first_sent_tok = tokenizer.tokenize(first_sent_lower)
    filtered_first_sent = [w for w in first_sent_tok if w not in stopwords.words('english')]
    
    regex_non_word = re.compile(r'[^\W]')
    filtered_final = filter(regex_non_word.search, filtered_first_sent)

    women_dict_1sent[name] = list(filtered_final)
    
print(women_dict_1sent)

    

KeyboardInterrupt: 

In [ ]:
# women_names_both = list()
# counter = 0
# for woman in women_names :
#     if woman not in women_dict_1sent : 
#         print("JFDSLKJFLSDKJFLDSJFSD")

# print(len(set(women_names)))
# print(len(women_dict_1sent))
    

In [ ]:
def sumWords(text): 
    wordDict = dict()
    for word in text :
        if word in wordDict :
            wordDict[word] = wordDict[word] + 1
        else :
            wordDict[word] = 1
    return (wordDict)

In [ ]:
# create the global variable of what is equivalent to good_types
unique_word_lst = list()
for woman in women_dict_1sent :
    summary = women_dict_1sent[woman]
    for word in summary :
        if word not in unique_word_lst :
            unique_word_lst.append(word)
        

In [ ]:
# creates a np array where the rows are women according to the list of women names
# columns are rows according to the list of unique words
def create_word_freq_array(input_women_dict_1sent, input_women_names, input_num_women, input_unique_word_lst):
    dict_freq = dict()

    for woman in input_women_dict_1sent :
        dict_freq[woman] = sumWords(input_women_dict_1sent[woman])
    
    np_with_freq = np.zeros(shape = (len(dict_freq), len(input_unique_word_lst)))
    i = 0
    for woman in input_women_names :
#         print(woman)
        if woman in dict_freq :
            j = 0
            for word in input_unique_word_lst :
                if word in dict_freq[woman] :
#                     print(j)
                    np_with_freq[i][j] = dict_freq[woman][word]
                j = j + 1
            i = i + 1
#     print(np_with_freq[0][0])
    return np_with_freq
#     print(input_unique_word_lst.index('chogha'))
    
        

In [ ]:
word_freq_array = create_word_freq_array(women_dict_1sent, women_names, len(women_dict_1sent), unique_word_lst)

In [16]:
# The file is ordered from the homeworks that we've worked on
# so that we can easily go back to the specific assignment if we need to

In [17]:
# TODO: Need to account for a woman's profile mentioning another woman's name

def create_j_sim_mat_women(input_num_women, input_word_freq_array, input_unique_word_lst):
    arr = np.zeros(shape = (input_num_women, input_num_women))
    
    for (i, woman1) in enumerate(input_word_freq_array) :
        for (j, woman2) in enumerate(input_word_freq_array) :
            s1 = np.nonzero(woman1)
            s2 = np.nonzero(woman2)
            intersect = np.intersect1d(s1, s2)
            union = np.union1d(np.array(s1).flatten(), np.array(s2).flatten())
            if len(union) > 0 :
                arr[i][j] = len(intersect)/len(union)
                
    np.fill_diagonal(arr, 0)
    
#     print(np.sum(arr[100:]))
    return arr
    
    

In [18]:
j_sim_mat_women = create_j_sim_mat_women(len(women_names), word_freq_array, unique_word_lst)

In [19]:
import operator

# make dictionary of top 5 women for each woman
def create_top_5_dict_women(j_sim_mat, num_women):
    women_top5 = {}
    # Loop through each woman
    for woman in range(num_women):
        top_5 = []      
        min_similarity = (-1, -1)
                
        for i in range(num_women):
            if (len(top_5) < 5): # Can just add bc we don't have top 5 yet
                top_5.append((i, j_sim_mat[woman][i])) # Stores (index, similarity)
                min_similarity = min(top_5, key = lambda t: t[1]) # Grabs tuple with minimum value
            else: # Only add similarity if it is greater than the minimum similarity
                if (j_sim_mat[woman][i] > min_similarity[1]):
                    top_5.remove(min_similarity) # Remove minimum similarity tuple
                    top_5.append((i, j_sim_mat[woman][i])) # Stores new (index, similarity)
                    min_similarity = min(top_5, key = lambda t: t[1]) # Grabs tuple with minimum value
                    
        # Sort tuple list of top 5
        top_5.sort(key = operator.itemgetter(1))
        # Descending order
        top_5.reverse()
        
        top_5_names = []
        
        for k, v in top_5:
            top_5_names.append(women_names[k])
            
        # Store woman (key) and list of top 5 most similar (value)
        women_top5[women_names[woman]] = top_5_names
        
    return(women_top5)
    

In [20]:
# make dictionary of top 5 women for each word
def create_top_5_dict_word(word_freq_array, num_words, num_women):
    word_top5 = {}
    # Loop through each word
    for word in range(len(word_freq_array.T)): # Iterates through words first
        top_5 = []      
        min_similarity = (-1, 0)
                
        for i in range(num_women):
            if (word_freq_array.T[word][i] > min_similarity[1]):
                if (len(top_5) < 5):
                    top_5.append((i, word_freq_array.T[word][i])) # Stores new (index, word count)
                    min_similarity = min(top_5, key = lambda t: t[1]) # Grabs tuple with minimum value
                else:
                    top_5.remove(min_similarity) # Remove minimum similarity tuple
                    top_5.append((i, word_freq_array.T[word][i])) # Stores new (index, word count)
                    min_similarity = min(top_5, key = lambda t: t[1]) # Grabs tuple with minimum value
                    
        # Sort tuple list of top 5
        top_5.sort(key = operator.itemgetter(1))
        # Descending order
        top_5.reverse()
        
        top_5_names = []
        
        for k, v in top_5:
            top_5_names.append(women_names[k])
            
        # Store word (key) and list of top 5 most similar (value)
        word_top5[unique_word_lst[word]] = top_5_names
        
    return(word_top5)

In [26]:
top_5_dict_women = create_top_5_dict_women(j_sim_mat_women, len(women_names))
top_5_dict_words = create_top_5_dict_word(word_freq_array, len(unique_word_lst), len(women_names))
pickle.dump(top_5_dict_women, open( "top_5_dict_women.pickle", "wb" ), protocol=2)
pickle.dump(top_5_dict_words, open( "top_5_dict_words.pickle", "wb" ), protocol=2)
print(top_5_dict_words)

{'english': ['Lilias Armstrong', 'Susan B. Horwitz', 'Mary Anning'], 'fossil': ['Mary Anning'], 'collector': ['Mary Anning'], 'dealer': ['Mary Anning'], 'paleontologist': ['Mary Anning'], 'became': ['Magda Peligrad', 'Helen F. Cullen', 'Grace Marie Bareis', 'Mildred Trotter', 'Leona Woods'], 'known': ['Laurette Séjourné', 'Mary Anning'], 'around': ['Phyllis Wise', 'Mary Anning'], 'world': ['Marion Elizabeth Stark', 'Anja Cetti Andersen', 'Katharina Galor', 'Margaret C. Snyder', 'Marie Curie'], 'important': ['Mary Anning'], 'finds': ['Helen Geake', 'Mary Anning'], 'made': ['Mary Anning'], 'jurassic': ['Mary Anning'], 'marine': ['Ulrike Beisiegel', 'Una Ryan', 'Antje Boetius', 'Marcia McNutt', 'Nancy Marcus'], 'beds': ['Mary Anning'], 'cliffs': ['Mary Anning'], 'along': ['Temperance "Bones" Brennan', 'Mary Anning'], 'channel': ['Mary Anning'], 'lyme': ['Mary Anning'], 'regis': ['Mary Anning'], 'county': ['Zsófia Torma', 'Ruth DeEtte Simpson', 'Harriet Cosgrove', 'Emer Jones', 'Mary Annin

In [22]:
# Homework 3 looked at Levenshtein distance (probably not a good idea imo)

In [23]:
def return_query(query, top_5_dict_words):
    if query in women_names: # EX: 'is similar to Mary Jackson'
        return(top_5_dict_women[query])
    else: # EX: 'worked at NASA'
        return(top_5_dict_words[query])

In [24]:
# Homework 4 looked at cosine similarity, which we can consider on top of jaccard

In [25]:
print(return_query('nasa', top_5_dict_words))

['Mary Jackson', 'Claudia Alexander']
